<a href="https://colab.research.google.com/github/Henil21/Twitter_Climate_Change/blob/main/Twitter_Climate_Change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Getting Data Ready 🌴

In [4]:
Data=pd.read_csv('/content/drive/MyDrive/twitter_sentiment_data.csv')

In [12]:
Data.head(5)
message=Data['message']
message[10]
sentiment=Data['sentiment']
print(f"message\n{message[10]}\n\nsentiment : {sentiment[10]}")


message
RT @DrDeJarnett: It's vital that the public health community addresses climate change- via @Climate4Health's Tabola #APHA2016 https://t.co/Ã¢â‚¬Â¦

sentiment : 1


## Visualizing random tweet 🎋